# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


####  Run this cell to set up and start your interactive session.


In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: b08c3312-3195-41f5-b426-20e9d6bb8950
Applying the following default arguments:
--glue_kernel_version 1.0.10
--enable-glue-datacatalog true
Waiting for session b08c3312-3195-41f5-b426-20e9d6bb8950 to get into ready status...
Session b08c3312-3195-41f5-b426-20e9d6bb8950 has been created.



Cargar bronce

In [ ]:
bronze_path = "s3://btc-imat-bucket/btc/"  
df = (spark.read
      .option("header", "true")
      .option("inferSchema", "true")
      .csv(bronze_path))

Ver si esta cargado

In [3]:
df.printSchema()
df.show(5, truncate=False)
print("Filas:", df.count())

root
 |-- date: date (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

+----------+---------+---------+---------+---------+---------+----+-----+
|date      |open     |high     |low      |close    |volume   |year|month|
+----------+---------+---------+---------+---------+---------+----+-----+
|2025-08-01|115778.44|116234.92|112792.32|113365.14|659.01624|2025|8    |
|2025-08-02|113365.14|114208.38|112104.48|112601.12|238.73724|2025|8    |
|2025-08-03|112605.84|115019.19|111971.32|114307.76|195.34322|2025|8    |
|2025-08-04|114311.02|115783.34|114080.28|115097.82|267.86495|2025|8    |
|2025-08-05|115119.48|115171.11|112707.47|114139.42|248.30182|2025|8    |
+----------+---------+---------+---------+---------+---------+----+-----+
only showing top 5 rows

Filas: 1461


Escribir el nivel silver

In [4]:
(df.write
 .mode("overwrite")
 .partitionBy("year", "month")
 .parquet("s3://btc-imat-bucket/btc_silver/"))